In [1]:
import pandas as pd
import os
from datetime import datetime
from util import *
from bs4 import BeautifulSoup

In [2]:
chain_df = pd.read_excel("chain.xlsx")
temp = chain_df[["name"]].value_counts()
print(temp[temp > 1])
protocol_df = pd.read_excel("protocol.xlsx")
temp = protocol_df[["name"]].value_counts()
print(temp[temp > 1])

protocol_df[protocol_df["name"] == "deBridge"]

Series([], dtype: int64)
name    
deBridge    2
dtype: int64


,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,oraclesByChain,pool2,assetToken,language,tokensExcludedFromParent,misrepresentedTokens,deadUrl,rugged,deadFrom,protocol
1060,1462,deBridge,NaN,-,https://debridge.finance,deBridge is a cross-chain interoperability and...,Multi-Chain,https://icons.llama.fi/debridge.jpg,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,debridge
4584,3846,deBridge,NaN,-,https://debridge.finance,The bridge that moves at lightspeed. Because D...,NaN,https://icons.llama.fi/debridge.jpg,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,debridge


In [3]:
protocol_df[protocol_df["category"].isnull()]

,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,oraclesByChain,pool2,assetToken,language,tokensExcludedFromParent,misrepresentedTokens,deadUrl,rugged,deadFrom,protocol


In [4]:
chain_list = chain_df.drop_duplicates(subset=["name"])
protocol_list = protocol_df.drop_duplicates(subset=["name"])

chain_list["code"] = chain_list["name"].apply(lambda x: x.replace(" ", "_"))
protocol_list["code"] = protocol_list["name"].apply(lambda x: x.lower().replace(" ", "-"))

chain_list = chain_list[["name", "code"]]
chain_list["category"] = None
protocol_list = protocol_list[["name", "code", "category"]]

df_list = pd.concat([chain_list, protocol_list], axis=0
    ).sort_values(by="name").reset_index(drop=True)

names = protocol_df["name"].values
df_list["url"] = df_list["name"].apply(lambda x: protocol_df[protocol_df["name"] == x]["url"].values[0] if x in names else None)
df_list.loc[df_list["name"] == "Acala", "url"] = "https://acala.network/"
df_list["start_date"] = None
df_list["end_date"] = None

df_list.shape

C:\Users\32mou\AppData\Local\Temp\ipykernel_28900\1118764949.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protocol_list["code"] = protocol_list["name"].apply(lambda x: x.lower().replace(" ", "-"))


(5069, 6)

In [5]:
l1 = [v.lower().replace("_", "-") for v in  os.listdir('df_chain')]
l2 = [v for v in os.listdir('df_protocol')]

dups_name = [
    protocol_df[protocol_df["protocol"]==v[:-4]]["name"].item() for v in set(l1).intersection(set(l2))
]

print(len(dups_name))

14


In [6]:
os.makedirs("final_tvl", exist_ok=True)

for f in os.listdir('df_chain'):
    df = pd.read_csv(f'df_chain/{f}')
    df["date"] = df["date"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    df["date"] = pd.to_datetime(df["date"]).dt.date
    temp = df["date"].value_counts()
    if temp[temp > 1].shape[0] > 0:
        print(f)
        print(temp[temp > 1])
        print(df[df["date"].isin(temp[temp > 1].index)])
    elif df[f[:-4]].sum() == 0:
        print(f, "Zeros")
    else:
        name = df_list[df_list["code"] == f[:-4]]["name"].values[0]
        if name in dups_name:
            continue
        df_list["start_date"][df_list["name"] == name] = df["date"].min().strftime('%Y-%m-%d')
        df_list["end_date"][df_list["name"] == name] = df["date"].max().strftime('%Y-%m-%d')
        df[name] = df[f[:-4]]
        df[["date", name]].to_csv(f'final_tvl/{name}.csv', index=False)

for f in os.listdir('df_protocol'):
    df = pd.read_csv(f'df_protocol/{f}')
    df["date"] = df["Timestamp"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df["date"] = pd.to_datetime(df["date"]).dt.date
    temp = df["date"].value_counts()
    if temp[temp > 1].shape[0] > 0:
        print(f)
        print(temp[temp > 1])
        print(df[df["date"].isin(temp[temp > 1].index)])
    elif df[f[:-4]].sum() == 0:
        print(f, "Zeros")
    else:
        name = df_list[df_list["code"] == f[:-4]]["name"].values[0]
        if name in dups_name:
            continue
        df_list["start_date"][df_list["name"] == name] = df["date"].min().strftime('%Y-%m-%d')
        df_list["end_date"][df_list["name"] == name] = df["date"].max().strftime('%Y-%m-%d')
        df[name] = df[f[:-4]]
        df[["date", name]].to_csv(f'final_tvl/{name}.csv', index=False)

In [7]:
temp = df_list.value_counts("name")
temp = temp[temp > 1].index
for name in temp:
    t = df_list[df_list["name"] == name]
    df_list.loc[df_list["name"] == name, "category"] = t[t["category"].notnull()]["category"].values[0]

,name,code,category,url,start_date,end_date
48,AILayer,AILayer,None,None,2024-06-24,2024-08-25
55,ALV,ALV,None,None,2023-08-26,2024-08-25
95,Aeternity,Aeternity,None,None,2024-07-17,2024-08-25
113,AirDAO,AirDAO,None,None,2024-01-16,2024-08-25
132,Alephium,Alephium,None,None,2023-11-22,2024-08-25
...,...,...,...,...,...,...
4921,dYdX,dYdX,None,None,2024-03-22,2024-08-25
4965,inEVM,inEVM,None,None,2024-03-12,2024-08-25
4987,opBNB,opBNB,None,None,None,None
4997,re.al,re.al,None,None,None,None


In [10]:
df_list.loc[df_list["category"].isnull(), "category"] = "Chain(s)"

df_list = df_list.drop_duplicates(subset=["name", "category"])    

print(df_list["category"].isnull().sum())
v = df_list.value_counts("name")
print(v[v > 1])

print(df_list.shape, len(os.listdir("final_tvl")))
df_list = df_list[['name', 'category', 'url', 'start_date', 'end_date']]
df_list.to_csv("final_tvl.csv", index=False)
df_list.head()

0
Series([], dtype: int64)
(5031, 6) 4402


,name,category,url,start_date,end_date
0,0.exchange,Dexes,https://app.0.exchange/,2021-02-03,2024-08-25
1,01,Derivatives,https://01.xyz/,2022-03-12,2024-08-25
2,0vix,Lending,https://www.0vix.com,2022-04-05,2023-12-14
3,0x,DEX Aggregator,https://www.0x.org/,None,None
4,0x Aggregator,DEX Aggregator,https://www.0x.org/,None,None


In [11]:
df_list.notnull().sum()

name          5031
category      5031
url           4761
start_date    4402
end_date      4402
dtype: int64